In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

import time
from datetime import datetime, timedelta
from pylib.tensorboardcmd import tensorboard_cmd

In [ ]:
#Note: increase this to at least 10 for a useful run, 20 or more produces better results
num_epochs = 10

<!-- requirement: images/VAE.png -->
<!-- requirement: images/VAEscheme.png -->
<!-- requirement: pylib/__init__.py -->
<!-- requirement: pylib/tensorboardcmd.py -->
<!-- requirement: pylib/tf_utils.py -->
<!-- requirement: pylib/mnist_dataset.py -->

# Variational Autoencoders


## Autoencoders


Autoencoders are neural networks where the number of input and output neurons are the same. If our input neurons represent pixels in an image, the output of our autoencoder will ideally be the input image. Why would we want to create a model that simply reproduces our data? If we restrict the number of neurons in our hidden layers to be less than the number of input or output neurons, we force our model to learn sparse representations of the data. Therefore, autoencoders can be used for image compression and removing noise from images. 

![VAE](images/VAE.png)

An autoencoder consists of two neural networks -- an **encoder** and **decoder**. The encoder takes in high dimensional data and generates low dimensional representations of that data. Then, the decoder takes the low dimensional representations and translates them back into the high dimensional input space. 


## Variational Autoencoders (VAEs)


Variational Autoencoders (VAEs) differ from regular autoencoders, because they not only learn sparse representations of data but also generate new data. Consequently, VAEs are used to [create new images](https://openai.com/blog/generative-models/). For example, we can train a VAE on the MNIST data set and have it create an image of a handwritten "5." 

How do VAEs generate new data? They do so by making smart assumptions about the distributions of these sparse data representations, or **latent vectors**. More generally, they belong to a class of models called generative models, which learn the joint probability distribution between the input ($x$) and output (or latent vectors, $z$). We can then use this information to come up with likely $(x,z)$ pairs. For example, once we learn the distribution corresponding to the sparse representation of a handwritten "5," we can sample from this distribution to form new latent vectors for "5." 

In this notebook, we will first build a simple autoencoder to recreate images in the MNIST data set and then will we build a VAE to generate new images of handwritten digits.

## Building an Autoencoder

We will start off as usual by loading our data and setting our variables.

In [ ]:
from pylib.tf_utils import mnist_test, mnist_train
train_images, train_label = mnist_train()
test_images, test_label = mnist_test()

In [ ]:
# Set parameters
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

batch_size = 128

In the code below, our encoder and decoder are neural networks (of two layers each) that are mirror images of each other. We feed the output of the encoder directly into the decoder to make the full autoencoder.

We want to allow the flexibility to change the number of layers and their size without much work, so we'll build the layers up with a for loop through a list of sizes.

Note that we are using the functional interface for this.  This allows us to build two distinct models, the encoder and decoder, then build our full model by using the former as the input to the latter.  This way, when we train the big model, we train both the decoder and encoder without extra work.

In [ ]:
#We'll build a network that goes 784 -> 256 -> 128 -> 256 -> 784
layer_sizes = [256, 128]

original = keras.layers.Input(shape=(img_size_flat,))

#Build the forward part, 784 -> 256 -> 128
layer = original
for size in layer_sizes:
    layer = keras.layers.Dense(size, activation='sigmoid')(layer)
#This is our encoder  
encoder_out = layer

encoder = keras.models.Model(original, encoder_out)
#encoder.summary()

#The decoder will be the reverse, 128 -> 256 -> 784
#We'll need to reverse our layers, 
#drop the first one (it's input now), and add the final shape
encoded_input = keras.layers.Input(shape=layer_sizes[-1:])
layer = encoded_input
for size in layer_sizes[-2::-1] + [img_size_flat]:
    layer = keras.layers.Dense(size, activation='sigmoid')(layer)
decoder_out = layer

decoder = keras.models.Model(encoded_input, decoder_out)
#decoder.summary()
    
autoencoder_out = decoder(encoder(original))
autoencoder = keras.models.Model(original, autoencoder_out)

## Adam optimizer


We want to minimize the loss due to inaccurate pixel values, so the loss function that we will use that penalizes these errors is mean-square-error.

Unlike last time, we will use the Adam Optimizer instead of the Gradient Descent Optimizer. This optimizer uses a decaying learning rate. 

As a reminder, we can interpret the learning rate as the size of the step we take down a gradient of our loss function. If the step size is too large, we may never get to the minimum. A large learning rate will manifest itself as noise in our loss curve that never converges to a minimum point. However, if we have a very small step size, our model may take a long time to run. Ideally, we want to take large steps at the start of the training process and small steps towards the end. The [Adam Optimizer](https://arxiv.org/pdf/1412.6980v8.pdf) changes the learning rate for us. 

In [ ]:
autoencoder.compile(loss='mse', optimizer='adam')

Then we will train our model.  Again, we note that our input and output are the same - we're training a model that reproduces its input.

In [ ]:
autoencoder.fit(train_images, train_images, epochs=num_epochs, 
                batch_size=batch_size, validation_data=(test_images,test_images))

Finally we will regenerate 10 of the MNIST images. We will display the original test images above the regenerated ones. 

In [ ]:
n_examples = 10

# Applying encode and decode over test set
encode_decode = autoencoder.predict(test_images[:n_examples])

# Compare original images with their reconstructions
f, a = plt.subplots(2, n_examples, figsize=(20, 4))
for i in range(n_examples):
    a[0][i].imshow(np.reshape(test_images[i], img_shape))
    a[1][i].imshow(np.reshape(encode_decode[i], img_shape))

## Application: Noise removal

Our system now has the capacity to reproduce the original images, and thus has some sort of internal model of what they should look like.  We can take advantage of this by feeding it images that _almost_ look like what it's seen before - it will return its best guess as to what they should be, given what it's seen.  Since we've trained it on clean images, we expect it will return clean images.  So, if we feed it a noisy image, it should do its best to return something like it's seen before, often very successfully removing the noise.

Here we'll take an image, add a uniform random number to a percentage of the pixels, then run it through the encode-decode pathway.  We'll also run the clean image through the pathway to compare.

In [ ]:
import random
def uniform_noise(img, percent):
    start = test_images[img].copy()
    noisy = start.copy()
    for i in range(len(test_images[img])):
        if random.random() < percent:
            noisy[i] += random.random()
            noisy[i] = min(noisy[i],1)
    f, a = plt.subplots(1, 4, figsize=(20, 4))
    encode_decode = autoencoder.predict(np.array([start, noisy]))
    a[0].imshow(np.reshape(start, img_shape))
    a[0].set_title("Input")
    a[0].grid(False)
    a[1].imshow(np.reshape(encode_decode[0], img_shape))
    a[1].set_title("Encode-decode on input")
    a[1].grid(False)
    a[2].imshow(np.reshape(noisy, img_shape))
    a[2].set_title("Noisy")
    a[2].grid(False)
    a[3].imshow(np.reshape(encode_decode[1], img_shape))
    a[3].set_title("Encode-decode on noisy")
    a[3].grid(False)

Here we've made a little function to do just that, just give it which test image (there are $10000$ of them) and what percentage of the pixels to add noise to.

In [ ]:
from ipywidgets import interact
interact(uniform_noise, img=(0,100), percent=(0,0.4,0.05));

## Generating new images

In order to generate new images we need to construct a **Variational Autoencoder**. With a simple autoencoder that we just constructed, we can not create new images since we don't know how to create latent vectors other than obtain them by encoding training images. In other words, we want to build a model that can generate images and not just memorize those from the training data set.  

As mentioned above, during training VAEs learn the **distribution** of latent variables so that we can then sample latent vectors from this distribution and decode them into new examples of handwritten digits. Therefore, we will now consider VAEs from a probability framework. 

Recall **Bayes' Theorem** that tells us:

$$\begin{eqnarray}
P(z \mid x) &=& \frac{P(x \mid z) \cdot P(z)}{P(x)} \\
\\
\text{posterior distribution} &=& \frac{\text{likelihood} \times \text{prior}}{\text{marginal likelihood}}
\end{eqnarray}$$


Say we have data $x$ and latent variables $z$. The **encoder** tries to approximate the posterior distribution $P(z \mid x)$, or generate latent variables conditioned on the data. On the other hand, the **decoder** takes $z$ [sampled from $P(z \mid x)$] and outputs parameters to the likelihood distribution $P(x \mid z)$. These parameters are the weights and biases of the neural networks. 


## KL-Divergence


The posterior distribution $P(z \mid x)$, which is what we are essentially looking for is often intractable. A way around that is to approximate it with a multivariate Gaussian $\mathcal{N}(\mu, \sigma)$ that approximates the true posterior distribution as best as possible. The amount of information lost when approximating $P(z \mid x)$ is called the [Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback–Leibler_divergence), and we will use it to construct our loss function. A choice commonly made to make our lives simple, is to have the true posterior distribution be a unit normal distribution and [calculate](http://allisons.org/ll/MML/KL/Normal/) the divergence accordingly:

$$
D_{\rm{KL}}(\mathcal{N}(0, 1) || \mathcal{N}(\mu, \sigma)) = 0.5*(\mu^2 + \sigma^2 - log(\sigma^2) - 1)
$$

This constraint on latent variables to have to approximate a certain distribution can also be thought of as a form of   regularization where we lose some fidelity to ensure we are capturing only important features. A nice explanation of choosing latent variables can be found [here](http://kvfrans.com/variational-autoencoders-explained/).

We also want to minimize the loss due to inaccurate pixel values when decoding images from latent space and must therefore create a component of the loss function that penalizes these errors. This component is often called generation loss, and it is the mean squared difference between the true and regenerated pixel values in our images. Total loss is then the sum of the mean squared error and Kullback-Leibler divergence:

$$
\mathcal{L} = MSE + D_{\rm{KL}}(\mathcal{N}(0, 1) || \mathcal{N}(\mu, \sigma))
$$

## Building a Variational Autoencoder

The following scheme represents the architecture of a variational autoencoder. Same as before, we will implement it using the functional interface.

![VAE scheme](images/VAEscheme.png)

Instead of working with the standard deviation of the approximate posterior distribution, we will be computing the $log(\sigma^2)$ because this is not only more convenient to work with, but also helps with numerical stability. A sample latent vector $z$ is then obtained as

$$
z = \mu + exp(0.5*log(\sigma^2)) * \epsilon
$$

where $\epsilon$ is a random normal tensor.
We choose both the encoder and decoder networks to have a single layer of 512 neurons and the size of the latent space to be 2.

In [ ]:
intermediate_dim = 512
latent_dim = 2

# input layer
x = keras.layers.Input(shape=(img_size_flat,))

# encoder network setup
h = keras.layers.Dense(intermediate_dim, activation='relu')(x)

# mean and std vector
z_mu = keras.layers.Dense(latent_dim)(h)
z_log_var = keras.layers.Dense(latent_dim)(h)

# sampling layer
def sampling(args):
    z_mu, z_log_var = args
    batch = keras.backend.shape(z_mu)[0]
    epsilon = keras.backend.random_normal(shape=(batch, latent_dim),
                              mean=0., stddev=1.)
    return z_mu + keras.backend.exp(0.5*z_log_var) * epsilon

z = keras.layers.Lambda(sampling)([z_mu, z_log_var])

# instantiate encoder model
encoder = keras.Model(x, [z_mu, z_log_var, z])

# decoder network setup
latent_inputs = keras.layers.Input(shape=(latent_dim,))
h = keras.layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = keras.layers.Dense(img_size_flat, activation='sigmoid')(h)

# instantiate decoder model
decoder = keras.Model(latent_inputs, outputs)


Now we can construct the full variational autoencoder and train it using the custom loss function.

In [ ]:
# variational autoencoder
outputs = decoder(encoder(x)[2])
vae = keras.Model(x, outputs)

def vae_loss(x, x_decoded):
    mse = tf.reduce_sum(tf.square(x - x_decoded))
    kl_loss = - 0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mu) - tf.exp(z_log_var), axis=-1)
    return mse + kl_loss

vae.compile(optimizer='adam', loss=vae_loss)

In [ ]:
vae.fit(train_images, train_images,
        shuffle=True,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=(test_images, test_images))

Now we can generate new images of handwritten digits. We chose our latent space to have 2 dimensions so let's now scan this latent plane by sampling points at regular intervals. From each latent vector obtained this way we will generate the corresponding digit by passing it through the decoder/generator network.

In [ ]:
n = 8  # compute n x n digits
figure = np.zeros((img_size * n, img_size * n))
# sample points within [-10, 10] standard deviations
grid_x = np.linspace(-10, 10, n)
grid_y = np.linspace(-10, 10, n)

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = decoder.predict(z_sample)
        digit = x_decoded[0].reshape(img_size, img_size)
        figure[i * img_size: (i + 1) * img_size,
               j * img_size: (j + 1) * img_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure)
plt.show()

## Exercise: New numbers

Take a different approach to generating new numbers: try creating a few new numbers as combinations of the old ones (they don't have to be linear combinations of two of them, you could do any size set, but remember that each pixel must be between 0 and 1).

## Exercise: Different compression

We have chosen a hidden size of 2 and it does very well.  Try playing with this size and see what happens if you change it.

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*